# Random Forests 5

---

__This Notebook__



__Results__ 



---

## Setup & Load

The are the results of the grid searches in the previous 3 notebooks (__12_RandomForests3.1__, __3.2__, and __3.3__).

In [1]:
import re
import os
import time
import joblib 

import numpy as np
import pandas as pd

from datetime import datetime

dt_object = datetime.fromtimestamp(time.time())
day, T = str(dt_object).split('.')[0].split(' ')
print('Revised on: ' + day)

Revised on: 2021-01-09


In [2]:
# load grid searches
mod_path = os.path.join("data","3_modeling")
filename = '01092021_rf_gridsearches.joblib'
                
gridsearches = joblib.load(os.path.join(mod_path, filename))

In [3]:
def extract_df(dic):
    df = pd.concat([
                    pd.DataFrame({'representation':[dic['representation']] \
                                * len(dic['gridsearch_res'].cv_results_["params"])}),
                    pd.DataFrame(dic['gridsearch_res'].cv_results_["params"]),
                    pd.DataFrame(dic['gridsearch_res'].cv_results_["mean_test_acc"], 
                                 columns=["mean_val_acc"]),
                    pd.DataFrame(dic['gridsearch_res'].cv_results_["mean_test_tpr"], 
                                 columns=["mean_val_tpr"]),
                    pd.DataFrame(dic['gridsearch_res'].cv_results_["mean_test_tnr"], 
                                 columns=["mean_val_tnr"])
                    ], axis=1)
    return df

# create list of dfs
df_list = []
for ix, dic in enumerate(gridsearches):
    df_list.append(extract_df(dic))

# flatten and reindex
dfm = pd.concat(df_list)
dfm.index = range(len(dfm))

# sort by top mean validation sensitivity
top_tpr = dfm.sort_values(by=['mean_val_tpr'], ascending=False).iloc[:4,:].copy()
top_tpr

,representation,max_depth,max_features,min_samples_split,n_estimators,mean_val_acc,mean_val_tpr,mean_val_tnr
67,X_bot_cossim_bot,20,500,5,200,0.991110,0.943320,0.998421
23,X_bot_cossim_bot,5,500,15,200,0.990766,0.943252,0.998027
21,X_bot_cossim_bot,5,500,10,200,0.990766,0.943252,0.998027
66,X_bot_cossim_bot,20,500,5,100,0.990767,0.940688,0.998421


In [4]:
dfm[dfm['representation']=='X_tfidf_cossim_tfidf'].sort_values(by=['mean_val_tpr'], ascending=False).iloc[:4,:]

,representation,max_depth,max_features,min_samples_split,n_estimators,mean_val_acc,mean_val_tpr,mean_val_tnr
135,X_tfidf_cossim_tfidf,20,250,10,200,0.986319,0.919973,0.99645
143,X_tfidf_cossim_tfidf,20,500,15,200,0.985976,0.917409,0.99645
90,X_tfidf_cossim_tfidf,5,500,5,100,0.985976,0.917409,0.99645
112,X_tfidf_cossim_tfidf,10,250,15,100,0.985976,0.917409,0.99645


In [5]:
import custom.old_gridsearch as og
import scipy.sparse as sp

# load target
raw_path = os.path.join("data","1_raw")
filename = "y_train.csv"
y = pd.read_csv(os.path.join(raw_path, filename))
y = np.array(y.iloc[:,0].ravel())
y[y=='ham'] = 0
y[y=='spam'] = 1
y = y.astype('int')

# load 12 matrices
proc_dir = os.path.join("data","2_processed")
Xnames = ['X_bot_svd_cos.npz', 'X_bot_tfidf_svd_cos.npz']
Xs = []
for ix, X in enumerate(Xnames):
    path_ = os.path.join(proc_dir, Xnames[ix])
    Xs.append(sp.load_npz(path_))

In [6]:
import custom.new_gridsearch as ng

param_grid = {
    'min_samples_split': [3, 5, 8],
    'n_estimators' : [300, 500],
    'max_depth': [20, 40],
    'max_features': [250, 500]
}

In [ ]:
grid_search_clf = ng.gridsearch_wrapper(
    Xs, Xnames, y, param_grid
)


Training 1: X_bot_svd_cos...
